# SECTION 1: Loading DataFrame

## 1) LOGIN

In [57]:
from google.colab import auth
auth.authenticate_user

print('Logged in!')

Logged in!


## 2) INISIASI CLIENT

In [58]:
from google.cloud import bigquery
import pandas as pd

## 3) QUERY TO DATAFRAME

In [59]:
PROJECT_ID = "workahola01"
client = bigquery.Client(project=PROJECT_ID)

In [63]:
SQL = """
SELECT duration_sec
FROM `bigquery-public-data.san_francisco_bikeshare.bikeshare_trips`
LIMIT 3000
"""

df = client.query(SQL).result().to_dataframe()
    # --> penggunaan syntax di atas menghasilkan error karena client.Query() butuh Application Default Credentials (ADC) aktif di environment.
    # --> Nah, auth.authenticate_user() login berhasil, namun tokennya tidak otomatis diregister ke ADC di semua env colab
    # Jadi agar cara ini berhasil, harus memiliki credential yang diregister ke ADC
    # --> google-cloud-bigquert client -> mencari tokrn di 'default credential store' -> gak ketemu., akhirnya error
    # CARA INI LEBIH DIUTAMAKAN

df = pd.read_gbq(SQL, project_id=PROJECT_ID, dialect='standard')
    # --> penggunaan pandas-gbq berhasil, karena langsung pakai token user yang aktif
    # CARA INI HANYA ALTERNATIF


/tmp/ipython-input-3461212466.py:14: FutureWarning: read_gbq is deprecated and will be removed in a future version. Please use pandas_gbq.read_gbq instead: https://pandas-gbq.readthedocs.io/en/latest/api.html#pandas_gbq.read_gbq
  df = pd.read_gbq(SQL, project_id=PROJECT_ID, dialect='standard')


In [64]:
df

,duration_sec
0,246
1,13010
2,952
3,1039
4,451
...,...
2995,1979
2996,463
2997,408
2998,409


In [65]:
type(df)

pandas.core.frame.DataFrame

In [62]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3000 entries, 0 to 2999
Data columns (total 1 columns):
 #   Column        Non-Null Count  Dtype
---  ------        --------------  -----
 0   duration_sec  3000 non-null   Int64
dtypes: Int64(1)
memory usage: 26.5 KB


# SECTION 2: Descriptive Statistics

## Perhitungan Central Tendency

In [70]:
mean_duration = df['duration_sec'].mean()
median_duration = df['duration_sec'].median()
modus_duration = df['duration_sec'].mode()[0]
modus_count = (df['duration_sec'] == modus_duration).sum()


print(f'Mean: {mean_duration:.2f}')
print(f'Median: {median_duration:.2f}')
print(f'Modus: {modus_duration:.2f} sebanyak: {modus_count} kali kemunculan')



Mean: 854.41
Median: 519.00
Modus: 427.00 sebanyak: 12 kali kemunculan


Insight:


*   Nilai mean (854.41 detik) jauh lebih besar dibandingkan nilai median (519 detik)
    - Mengindikasikan distribusi adalah positive skew, artinya ada sebagian kecil perjalanan yang sangat lama, sehingga menarik rata-rata ke kanan.
    - Jadi jika rata-rata peminjam sepeda menghabiskan waktu sekitar 10-15 menit, ada sebagian kecil yang meminjam jauh lebih lama dari waktu ini.
*   Nilai modus (427 detik) menunjukkan bahwa durasi paling umum/paling banyak berada di kisaran 7 menitan.
    - Mengindikasikan bahwa pola peminjaman sepeda banyak di durasi singkat, bukan long trip.
*   Jika disimpulkan, distribusi data cenderung tidak simetris, dan median lebih representatif untuk menggambarkan perilaku pengguna rata-rata yang sesungguhnya.





## Pengecekan Dispersi

In [52]:
range_duration = df['duration_sec'].max() - df['duration_sec'].min()
variance_duration = df['duration_sec'].var()
stdev_duration = df['duration_sec'].std()

print(f'Range: {range_duration:.2f}')
print(f'Variance: {variance_duration:.2f}')
print(f'Standard Deviation: {stdev_duration:.2f}')

Range: 82081.00
Variance: 7518231.20
Standard Deviation: 2741.94


Insight:


*   Range mencapai 22.8 jam. Artinya, perbedaan antara perjalanan tercepat dan terlama mencapai hampir 23 jam. Dengan kata lain, di dataset ini, ada pengguna yang menyewa hampir 1 hari penuh, sementara ada sebagian yang menyewa selama beberapa menit saja.
*   Variance mengukur seberapa jauh data tersebar dari rata-ratanya. Nilai yang sangat besar ini mengindikasikan bahwa penyebaran durasi sangatlah tinggi.
*   Standar deviasi adalah sekitar 45.7 menit, yang artinya dalah setiap perjalanan memiliki perbedaan sekitar 45 menit dari mean (14 menit). Ini menandakan distribusi sangat lebar dan tidak stabil (ada bayak nilai ekstrim di dalam dataset ini)
Nilai dispersi yang sangat tinggi (range ±82 ribu detik dan standar deviasi ±2.742 detik) menunjukkan bahwa durasi perjalanan pelanggan bervariasi sangat luas. Sebagian besar pelanggan melakukan perjalanan singkat (~7–9 menit), namun terdapat sebagian kecil pelanggan dengan durasi sangat panjang hingga puluhan ribu detik yang secara signifikan memperlebar sebaran data.
*   Dengan kondisi ini, standar deviasi hampir 3× lipat lebih besar dari rata-rata, menandakan distribusi data sangat tidak homogen dan kemungkinan besar mengandung banyak outlier ekstrem. Oleh karena itu, median lebih representatif daripada mean untuk menggambarkan “durasi tipikal” pelanggan.




## Pengecekan Skewness dan Kurtosis

In [53]:
skewness_duration = df['duration_sec'].skew()
kurtosis_duration = df['duration_sec'].kurtosis()

print(f'Skewness: {skewness_duration:.2f}')
print(f'Kurtosis: {kurtosis_duration:.2f}')

Skewness: 17.95
Kurtosis: 401.75


Inisght:

Hasil perhitungan menunjukkan nilai skewness sebesar 17.95, yang mengindikasikan distribusi data sangat condong ke knaan (extremely right-skewed). Artinya, sebagian besar pelanggan melakukan perjalanan berdurasi singkat, sementara terdapat sejumlah kecil perjalanan dengan durasi sangat panjang yang menarik distribusi ke arah kanan.

Selain itu, nilai kurtosis sebesar 401.75 menandakan distribusi yang sangat runcing (leptokurtic) dan memiliki ekor kanan yang tebal. Ini berarti data sangat terkonsentrasi di sekitar nilai kecil (perjalanan singkat), namun terdapat beberapa nilai ekstrim (outlier) yang sangat jauh dari pusat distribusi.

Secara keseluruhan, pola ini memperkuat indikasi bahwa dataset mengandung banyak outlier ekstrem, dan langkah outlier detection adn handling sangat penting dilakukan sebelum melanjutkan analisis statistik lanjutan.


## Mendeteksi Outlier dengan Metode IQR (Tukey's Rule)

Mengapa menggunakan Tukey's Rule:
- Skewness = 17.95 → sangat right-skewed
- Kurtosis = 401.75 → ekor kanan sangat tebal
- Distribusi jauh dari normal

Sehingga Z-Score method menjadi tidak cocok karena Z-Score memakai asumsi data 'simetris' dan terpusat di mean.

In [77]:
Q1 = df['duration_sec'].quantile(0.25)
Q3 = df['duration_sec'].quantile(0.75)
IQR = Q3 - Q1

lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

outliers = df[(df['duration_sec'] < lower_bound) | (df['duration_sec'] > upper_bound)]

numbers_outliers = len(outliers)
numbers_total = len(df)
percentage_outliers = (numbers_outliers/numbers_total) * 100

print(f'Q1 (25%) {Q1:.2f}')
print(f'Q3 (75%) {Q3:.2f}')
print(f'IQR: {IQR:.2f}')
print(f'Batas bawah: {lower_bound:.2f}')
print(f'Batas atas: {upper_bound:.2f}')
print(f'Jumlah outlier: {numbers_outliers}')
print(f'Persentase outlier: {percentage_outliers:.2f}')

Q1 (25%) 319.00
Q3 (75%) 807.25
IQR: 488.25
Batas bawah: -413.38
Batas atas: 1539.62
Jumlah outlier: 176
Persentase outlier: 5.87


Insight:

Berdasarkan analisis IQR, diperoleh Q1 = 338 detik dan Q3 = 863 detik degnan IQR sebesar 525 detik. Dengan demikian, batas atas untuk mendeteksi outlier adalah sekitar 1.651 detik (≈27 menit). Data yang memiliki durasi lebih dari nilai ini dikategorikan sebagai outlier.

Dari total 3.000 observasi, terdapat sekitar 176 data (5,8%) yang dikategorikan sebagai outlier, seluruhnya berada di sisi kanan distribusi. Hal ini sejalan dengan hasil skewness dan kurtosis sebelumnya yang menunjukkan distribusi sagnt condong ke kanan dan memiliki ekor kanan tebal.

Keberadaan outlier ini menandakan adanya sejumlah kecil pelanggan dengan durasi sewa sangat panjang, yang kemunkinan disebabkan oleh:

- pengguna yang melakukan perjalanan jauh atau long trip,

- pelanggan yang lupa mengembalikan sepeda,

- atau kesalahan pencatatan sistem waktu (data entry error).

Secara analitis, keberadaan outlier ini menyebabkan nilai rata-rata menjadi tidak representatif, sehingga median tetap menjadi ukuran pemusatan yang paling tepat untuk menggambarkan “durasi tipikal” pelanggan.

## The Difference Between Before and After Outliers Handling

In [83]:
Q1 = df['duration_sec'].quantile(0.25)
Q3 = df['duration_sec'].quantile(0.75)
IQR = Q3 - Q1
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

df_no_outlier = df[(df['duration_sec'] >= lower_bound) & (df['duration_sec'] <= upper_bound)]

mean_before = df['duration_sec'].mean()
median_before = df['duration_sec'].median()

mean_after = df_no_outlier['duration_sec'].mean()
median_after = df_no_outlier['duration_sec'].median()

print(f'Jumlah data awal       : {len(df)}')
print(f'Jumlah data setelah IQR : {len(df_no_outlier)}')
print()
print(f'Mean sebelum outlier dihapus  : {mean_before:.2f}')
print(f'Median sebelum outlier dihapus: {median_before:.2f}')
print()
print(f'Mean setelah outlier dihapus  : {mean_after:.2f}')
print(f'Median setelah outlier dihapus: {median_after:.2f}')

Jumlah data awal       : 3000
Jumlah data setelah IQR : 2824

Mean sebelum outlier dihapus  : 854.41
Median sebelum outlier dihapus: 519.00

Mean setelah outlier dihapus  : 558.37
Median setelah outlier dihapus: 492.00


Insight:

Setelah dilakukan penghapusan outlier menggunakan metode IQR, jumlah data berkurang dari 3.000 menjadi sekitar 2.855 observasi (±4,8% data termasuk outlier).

Nilai mean durasi perjalanan turun dari 854 detik (sekitar 14 menit) menjadi 642 detik (sekitar 10,7 menit), sedangkan median tetap stabil di sekitar 519 detik (sekitar 8,6 menit). Penurunan signifikan pada mean ini menunjukkan bahwa sebelumnya terdapat sejumlah kecil perjalanan berdurasi sangat panjang yang secara tidak proporsional menaikkan nilai rata-rata.

Setelah outlier dihapus, distribusi durasi perjalanan mejnadi lebih representatif terhadap perilaku mayoritas pelanggan, dengan median yang tetap menjadi ukuran pemusatan terbaik untuk menggambarkan “durasi tipikal” pengguna sepeda di San Francicso.

## Export to .csv

In [84]:
df_no_outlier.to_csv('bikeshare_trips_clean.csv', index=False)

Check apakah .csv sudah tersimpan di colab

In [86]:
!ls -lh /content

total 16K
-rw-r--r-- 1 root root  12K Nov  2 12:12 bikeshare_trips_clean.csv
drwxr-xr-x 1 root root 4.0K Oct 30 13:36 sample_data


Download .csv

In [88]:
from google.colab import files
files.download('bikeshare_trips_clean.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>